In [1]:
import numpy as np
import heapq

# Parsing

In [2]:
def parse(inlines):
    return np.array([[int(c) for c in l] for l in inlines.split("\n")])

with open("data/input17.txt") as f:
    data = f.read().rstrip("\n")

mat = parse(data)

# Part 1

In [3]:
def turn_right(curdir):
    if curdir==(0,1):
        return (1,0)
    if curdir==(0,-1):
        return (-1,0)
    if curdir==(1,0):
        return (0,-1)
    if curdir==(-1,0):
        return (0,1)

def turn_left(curdir):
    if curdir==(0,1):
        return (-1,0)
    if curdir==(0,-1):
        return (1,0)
    if curdir==(1,0):
        return (0,1)
    if curdir==(-1,0):
        return (0,-1)

def step(space, curstate):
    newstates = []
    dirleft = turn_left(curstate["dir"])
    dirright = turn_right(curstate["dir"])
    posleft = (curstate["pos"][0] + dirleft[0], curstate["pos"][1] + dirleft[1])
    posright = (curstate["pos"][0] + dirright[0], curstate["pos"][1] + dirright[1])
    posstraight = (curstate["pos"][0] + curstate["dir"][0], curstate["pos"][1] + curstate["dir"][1])
    if 0<=posleft[0]<space.shape[0] and 0<=posleft[1]<space.shape[1]:
        newstates.append({"pos": posleft, "dir": dirleft, "nstraight": 1})
    if 0<=posright[0]<space.shape[0] and 0<=posright[1]<space.shape[1]:
        newstates.append({"pos": posright, "dir": dirright, "nstraight": 1})
    if 0<=posstraight[0]<space.shape[0] and 0<=posstraight[1]<space.shape[1]:
        if curstate["nstraight"]<3:
            newstates.append({"pos": posstraight, "dir": curstate["dir"], "nstraight": curstate["nstraight"] + 1})
    return newstates

dir2ind = {(0,1): 0, (0,-1): 1, (1,0): 2, (-1,0): 3}
ind2dir = {v:k for k,v in dir2ind.items()}

class Node:
    def __init__(self, cost, ind):
        self.ind = ind
        self.cost = cost
    def __lt__(self, other):
        return self.cost < other.cost

def dijkstra(space, curdict, goalpos):
    nnodes = 0
    state2ind = {}
    state2ind[((0,0),0,0)] = 0
    nnodes = 1
    for i in range(space.shape[0]):
        for j in range(space.shape[1]):
            for m in range(4):
                for n in range(1,4):
                    state2ind[((i,j), m, n)] = nnodes
                    nnodes += 1
    ind2state = {v:k for k,v in state2ind.items()}

    q = []
    curstate = (curdict["pos"], dir2ind[curdict["dir"]], curdict["nstraight"])
    heapq.heappush(q, Node(0, state2ind[curstate]))
    amap = {state2ind[curstate]: 0}
    visited = [False]*nnodes
    visited[state2ind[curstate]] = True

    for i in range(nnodes):
        if not q:
            break
        curnode = heapq.heappop(q)
        while visited[curnode.ind]:
            if not q:
                break
            curnode = heapq.heappop(q)
        curind = curnode.ind
        curcost = curnode.cost
        curstate = ind2state[curind]
        visited[state2ind[curstate]] = True
        curdict = {"pos": curstate[0], "dir": ind2dir[curstate[1]], "nstraight": curstate[2]}
        for n in step(space, curdict):
            newstate = (n["pos"], dir2ind[n["dir"]], n["nstraight"])
            newind = state2ind[newstate]
            if visited[newind]==False:
                newcost = curcost + space[n["pos"][0],n["pos"][1]]
                if newind not in amap:
                    amap[newind] = newcost
                else:
                    if newcost<amap[newind]:
                        amap[newind] = newcost
                heapq.heappush(q, Node(newcost, newind))
    
    mincost = float('inf')
    for nodeind, nodecost in amap.items():
        if ind2state[nodeind][0]==goalpos:
            if nodecost<mincost:
                mincost = nodecost
    return mincost

curdict = {}
curdict["pos"] = (0,0)
curdict["dir"] = (0,1)
curdict["nstraight"] = 0
goalpos = (mat.shape[0]-1, mat.shape[1]-1)

cost = dijkstra(mat, curdict, goalpos)
print(cost)

855


# Part 2

In [4]:
def step(space, curstate):
    newstates = []
    dirleft = turn_left(curstate["dir"])
    dirright = turn_right(curstate["dir"])
    posleft = (curstate["pos"][0] + dirleft[0], curstate["pos"][1] + dirleft[1])
    posright = (curstate["pos"][0] + dirright[0], curstate["pos"][1] + dirright[1])
    posstraight = (curstate["pos"][0] + curstate["dir"][0], curstate["pos"][1] + curstate["dir"][1])
    if curstate["nstraight"]>=4 or curstate["nstraight"]==0:
        if 0<=posleft[0]<space.shape[0] and 0<=posleft[1]<space.shape[1]:
            newstates.append({"pos": posleft, "dir": dirleft, "nstraight": 1})
        if 0<=posright[0]<space.shape[0] and 0<=posright[1]<space.shape[1]:
            newstates.append({"pos": posright, "dir": dirright, "nstraight": 1})
    if 0<=posstraight[0]<space.shape[0] and 0<=posstraight[1]<space.shape[1]:
        if curstate["nstraight"]<10:
            newstates.append({"pos": posstraight, "dir": curstate["dir"], "nstraight": curstate["nstraight"] + 1})
    return newstates

def dijkstra(space, curdict, goalpos):
    nnodes = 0
    state2ind = {}
    state2ind[((0,0),0,0)] = 0
    nnodes = 1
    for i in range(space.shape[0]):
        for j in range(space.shape[1]):
            for m in range(4):
                for n in range(1,11):
                    state2ind[((i,j), m, n)] = nnodes
                    nnodes += 1
    ind2state = {v:k for k,v in state2ind.items()}

    q = []
    curstate = (curdict["pos"], dir2ind[curdict["dir"]], curdict["nstraight"])
    heapq.heappush(q, Node(0, state2ind[curstate]))
    amap = {state2ind[curstate]: 0}
    visited = [False]*nnodes
    visited[state2ind[curstate]] = True

    iter = 0
    for i in range(nnodes):
        if not q:
            break
        curnode = heapq.heappop(q)
        while visited[curnode.ind]:
            if not q:
                break
            curnode = heapq.heappop(q)
        curind = curnode.ind
        curcost = curnode.cost
        curstate = ind2state[curind]
        visited[state2ind[curstate]] = True
        curdict = {"pos": curstate[0], "dir": ind2dir[curstate[1]], "nstraight": curstate[2]}
        for n in step(space, curdict):
            newstate = (n["pos"], dir2ind[n["dir"]], n["nstraight"])
            newind = state2ind[newstate]
            if visited[newind]==False:
                newcost = curcost + space[n["pos"][0],n["pos"][1]]
                if newind not in amap:
                    if not (newstate[0]==goalpos and newstate[2]<4):
                        amap[newind] = newcost
                else:
                    if newcost<amap[newind]:
                        if not (newstate[0]==goalpos and newstate[2]<4):
                            amap[newind] = newcost
                heapq.heappush(q, Node(newcost, newind))
        iter += 1
    
    mincost = float('inf')
    for nodeind, nodecost in amap.items():
        if ind2state[nodeind][0]==goalpos:
            if nodecost<mincost:
                mincost = nodecost
    return mincost

curdict = {}
curdict["pos"] = (0,0)
curdict["dir"] = (0,1)
curdict["nstraight"] = 0
goalpos = (mat.shape[0]-1, mat.shape[1]-1)

cost = dijkstra(mat, curdict, goalpos)
print(cost)

980
